In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import ext_transforms as et
from dataset import SatelliteDataset
from torch.utils.data import DataLoader
from torchvision.io import write_png

In [2]:
model = model = torchvision.models.segmentation.deeplabv3_resnet50(num_classes=1)
model.load_state_dict(torch.load('best_resnet_model'))
model.eval()

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [3]:
val_transform = et.ExtCompose([
    et.ExtToTensor(),
    et.ExtNormalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225])
])

In [4]:
dataset = SatelliteDataset(image_dir='test_images', mask_dir='masks', transform=val_transform)
dataloader = DataLoader(dataset, batch_size=1)

convert  tensor to png

In [5]:
sigmoid = torch.nn.Sigmoid()
for i, data in enumerate(dataloader) :
    image, _ = data
    image = image.to(dtype=torch.float32)
    prediction = sigmoid(model(image)['out'][0]) > 0.5
    write_png(prediction.to(dtype=torch.uint8) * 255, 'inference_best_resnet_model_image_{}.png'.format(i))